In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
class GemmaRouter:
    """
    A simple wrapper for using Google's Gemma-2-2B-IT model
    via Hugging Face's OpenAI-compatible inference API.
    """

    def __init__(
        self,
        model: str = "google/gemma-2-2b-it:nebius",
        token_env: str = "HF_THIRD_TOKEN",
        base_url: str = "https://router.huggingface.co/v1",
    ):
        load_dotenv()
        api_key = os.getenv(token_env)
        if not api_key:
            raise ValueError(
                f"Missing token: environment variable '{token_env}' not found."
            )

        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model = model

    def invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Non-streaming response."""
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.choices[0].message.content.strip()

    def stream_invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Streaming response."""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            stream=True,
        )
        return stream
model=GemmaRouter()


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\91800\Desktop\GENAI UDEMY\genai-udemy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model2=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
from langchain.agents import initialize_agent,AgentType
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2,include_answer=True,include_images=True)
tools=TavilySearchResults(max_results=1,include_answer=True)

In [6]:
from langchain.agents import initialize_agent

In [9]:
# Create the agent
agent = initialize_agent(
    tools=[tools],
    llm=model2,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [10]:
agent.invoke("Give me a funny tweet about today's Pm scene")



> Entering new AgentExecutor chain...
Action: tavily_search_results_json
Action Input: "UK Prime Minister news today funny"
Observation: [{'url': 'https://www.youtube.com/watch?v=cCXAg1knidg', 'content': "Watch PMQs with British Sign Language (BSL) - https://youtube.com/live/gZ8LoNw7UmE?feature=share Prime Minister's Question Time,"}]
Thought:Action: tavily_search_results_json
Action Input: "Prime Minister's Questions today funny moments"
Observation: [{'url': 'https://www.youtube.com/playlist?list=PL40441042C458B62B', 'content': "PMQs LIVE: Prime Minister's Questions - 5 November 2025. UK Parliament · 37:20 · PMQs LIVE: Prime Minister's Questions - 29 October 2025. UK Parliament."}]
Thought:Thought: The previous searches for "UK Prime Minister news today funny" and "Prime Minister's Questions today funny moments" did not yield specific funny tweets or current humorous content. The results were primarily links to full PMQs sessions, some with future dates (e.g., Nov 2025), which are 

{'input': "Give me a funny tweet about today's Pm scene",
 'output': '"Rishi Sunak\'s election speech today was so impactful, even the sky started crying. Guess \'things can only get wetter\' isn\'t just a political slogan anymore. #Election2024 #RishiSunak"'}

In [31]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_openai import ChatOpenAI # or whichever model you use

# --- SETUP ---
# 1. The Tool (Tavily)
search_tool = TavilySearchResults(max_results=2)

# 2. The Model (The Brain) - Assuming 'model2' is your LLM from previous code
llm = model2 

# 3. The Agent (Brain + Tool)
agent = initialize_agent(
    tools=[search_tool], 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, handle_parsing_errors=True 
)

query = "What is the exact current stock price of Apple (AAPL) right now?"

print(f"--- TEST 1: STANDARD LLM (Brain Only) ---\n")
try:
    # The LLM has no internet. It relies on training data from years ago.
    response_1 = llm.invoke(query) 
    print(f"LLM Answer: {response_1.content}\n")
except Exception as e:
    print(f"LLM Answer: I can't answer that (My knowledge cutoff is in the past).\n")


print(f"--- TEST 2: TAVILY TOOL (Search Only) ---\n")
# The tool has internet, but no brain. It just dumps raw data.
response_2 = search_tool.invoke(query)
print(f"Tool Output: {response_2}\n")


print(f"--- TEST 3: AGENT (Brain + Tool) ---\n")
# The Agent thinks, uses the tool, reads the mess, and explains it to you.
response_3 = agent.invoke(query)
print(f"Agent Answer: {response_3['output']}")

--- TEST 1: STANDARD LLM (Brain Only) ---

LLM Answer: I cannot provide you with the exact, real-time current stock price of Apple (AAPL) right now. Stock prices fluctuate second by second throughout the trading day, and my knowledge base is not updated in real-time.

To get the most up-to-the-minute price, you would need to check a live financial source such as:

*   **Google Finance**
*   **Yahoo Finance**
*   **Bloomberg.com**
*   **Reuters.com**
*   **Your brokerage account or app**

These platforms will show you the live trading data.

--- TEST 2: TAVILY TOOL (Search Only) ---

Tool Output: [{'url': 'https://www.marketwatch.com/investing/stock/aapl?gaa_at=eafs&gaa_n=AWEtsqfVo1hjqVCXbWM-fwGM7MiKXlR8wJxiLhGeOAhR9oJrBEI0ZjUCVBuw&gaa_ts=691f0203&gaa_sig=eOevQSnb4YcEjj908ohUXP0x42lY90LZtyM7DUDpMn5hCYxpwrLsKCwDCPdaCWwS5mqWm2OACVneIvFJoAooHA%3D%3D', 'content': 'AAPL | Complete Apple Inc. stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview

In [29]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools.tavily_search import TavilySearchResults

# --- SETUP ---
search_tool = TavilySearchResults(max_results=2)
llm = model2 

# --- THE FIX IS HERE ---
agent = initialize_agent(
    tools=[search_tool], 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    # This tells the agent: "If the LLM output is messy, don't crash!"
    handle_parsing_errors=True 
)

query = "What is the exact current stock price of Apple (AAPL) right now?"

print(f"--- TEST 3: AGENT (Brain + Tool) ---\n")
try:
    response_3 = agent.invoke(query)
    print(f"Agent Answer: {response_3['output']}")
except Exception as e:
    print(f"Error: {e}")

--- TEST 3: AGENT (Brain + Tool) ---



> Entering new AgentExecutor chain...
Action: tavily_search_results_json
Action Input: current stock price of Apple AAPL
Observation: [{'url': 'https://finance.yahoo.com/quote/AAPL/history/', 'content': 'Volume. Nov 19, 2025, 265.53, 272.21, 265.50, 268.56, 268.56, 40,355,600. Nov 18, 2025, 269.99, 270.71, 265.32, 267.44, 267.44, 45,677,300.'}, {'url': 'https://finance.yahoo.com/quote/AAPL/', 'content': "Apple's stock has surged 15,308% over the past 20 years, but analysts predict a slowdown in growth. Recent sales of the iPhone 17 series in"}]
Thought:Action: tavily_search_results_json
Action Input: Apple AAPL latest stock price
Observation: [{'url': 'https://investor.apple.com/stock-price/default.aspx', 'content': "Stock Quote: NASDAQ: AAPL · Day's Open265.52 · Closing Price268.56 · Volume40.4M · Intraday High272.21 · Intraday Low265.50."}, {'url': 'https://finance.yahoo.com/quote/AAPL/news/', 'content': "News headlines Apple's stock has surged